In [ ]:
from transformers import MBartForConditionalGeneration
import torch
from transformers import AutoModel, AutoTokenizer
import time 
from tqdm import tqdm
bartpho = AutoModel.from_pretrained("vinai/bartpho-syllable")
bartpho = MBartForConditionalGeneration.from_pretrained("vinai/bartpho-syllable")
tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-syllable")
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import pipeline
pipeline = pipeline(
    task="fill-mask",
    model="vinai/phobert-base",
    torch_dtype=torch.float16,
    device=0
)
def remove_special_characters(text):
    import re
    # Remove special characters and digits, keep only Vietnamese characters and spaces
    text = re.sub(r'[^a-zA-ZÀ-ỹ\s]', '', text)
    return text.strip()
def topk_predictions(text, top_k=5):
    # input_ids = tokenizer([text], return_tensors="pt")["input_ids"]
    # logits = bartpho(input_ids).logits
    # masked_index = (input_ids[0] == tokenizer.mask_token_id).nonzero().item()

    # probs = logits[0, masked_index].softmax(dim=0)
    # values, predictions = probs.topk(top_k)
    
    # return tokenizer.decode(predictions).split()
    
    output = pipeline(text, top_k=top_k)
    mask_index = text.index('<mask>')
    # Extract the predictions from the output
    predictions = []
    for item in output:
        if 'token_str' in item:
            predictions.append(item['token_str'])
    
    predictions = [remove_special_characters(item['token_str']) for item in output]

def sentence_prediction(text, top_k=300):
    error_indices = []
    for i in range(len(text.split())):
        original_word
        new_list = text.split()
        # replace the i-th word with a mask token
        # clean_word = remove_special_characters(new_list[i])
        clean_word = new_list[i]
        new_list[i] = "<mask>"
        masked_text = " ".join(new_list)
        start_time = time.time()
        predictions = topk_predictions(masked_text, top_k=top_k)
        # for j in range(len(predictions)):
        #     predictions[j] = remove_special_characters(predictions[j])
        end_time = time.time()
        if clean_word not in predictions:
            error_indices.append((i, clean_word, predictions[0]))
            print(f"Masked Text: {masked_text}")
            print(f"->Top {top_k} Predictions: {predictions}")
            
        # print(f"Masked Text: {masked_text}")
    return error_indices

def paragraph_prediction(text, top_k=300):
    sentences = text.split('. ')
    all_errors = []
    for sentence in sentences:
        errors = sentence_prediction(sentence, top_k=top_k)
        if errors:
            all_errors.append((sentence, errors))
            
    return all_errors

In [55]:

TXT = "Trung tâm Dự báo khí tượng thủy văn quốc gia cho biết lúc 7h hôm nay, áp thấp nhiệt đới mạnh 61 km/h, cấp 6-7, "
TXT = "Theo thầy Đỗ Đình Đảo, Hiệu trưởng, những điều chỉnh này sẽ được thông báo cho học sinh vào ngày tựu trường 20/8 tới đây."
paragraph_prediction(TXT, top_k=100)

Masked Text: Theo thầy Đỗ Đình <mask> Hiệu trưởng, những điều chỉnh này sẽ được thông báo cho học sinh vào ngày tựu trường 20/8 tới đây.
->Top 100 Predictions: ['Th', 'T', 'C', 'Hùng', 'Ph', 'Ho', 'Trung', 'Phụ', 'Kh', 'Qu', 'Tuấn', 'L', 'Đức', 'Vinh', 'Khánh', 'Tr', 'Thuận', 'Hồng', 'H', 'Tiến', 'Việt', 'Sơn', 'Hà', 'Giao', 'Thắng', 'Thu', 'Ngh', 'Khoa', 'Hiệp', 'Khả', 'Bình', 'Diễn', 'Tu', 'Ki', 'Châu', 'Quý', 'S', 'Tài', 'Hu', 'Phi', 'Bá', 'D', 'Phong', 'Viên', 'Đ', 'Ng', 'Tân', 'Tâm', 'Hợp', 'Ngọc', 'Ch', 'Phúc', 'Thái', 'Á', 'Hưng', 'Phú', 'Thanh', 'Dũng', 'Â', 'Duy', 'Toàn', 'Quy', 'Liên', 'Dung', 'Phương', 'Long', 'Thảo', 'Nguyên', 'Chương', 'Hoàn', 'X', 'Trọng', 'Thị', 'M', 'Bí', 'Thi', 'Di', 'Hi', 'Thành', 'Độ', 'Minh', 'Tho', 'Loan', 'Văn', 'Tru', 'Quang', 'Hiển', 'V', 'Thân', 'Lợi', 'Bảo', 'B', 'Tuy', 'Anh', 'Tha', 'K', 'San', 'Hải', 'Chính']
Masked Text: Theo thầy Đỗ Đình Đảo, Hiệu <mask> những điều chỉnh này sẽ được thông báo cho học sinh vào ngày tựu trường 20/8 tới đây.


[('Theo thầy Đỗ Đình Đảo, Hiệu trưởng, những điều chỉnh này sẽ được thông báo cho học sinh vào ngày tựu trường 20/8 tới đây.',
  [(4, 'Đảo,', 'Th'),
   (6, 'trưởng,', 'trưởng'),
   (18, 'vào', 'và'),
   (20, 'tựu', 'khai'),
   (22, '20/8', '20,'),
   (23, 'tới', 'tới./2010,'),
   (24, 'đây.', '.')])]

In [52]:

#BARTpho-word
word_tokenizer = AutoTokenizer.from_pretrained("vinai/bartpho-word")
bartpho_word = AutoModel.from_pretrained("vinai/bartpho-word")


In [53]:
TXT = 'Trung tâm Dự báo khí tượng thủy văn quốc gia cho biết lúc 7h hôm nay, áp thấp nhiệt đới mạnh 61 km/h, cấp 6-7,'  
input_ids = word_tokenizer(TXT, return_tensors='pt')['input_ids']
features = bartpho_word(input_ids).logits


AttributeError: 'Seq2SeqModelOutput' object has no attribute 'logits'

In [ ]:
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")

# INPUT TEXT MUST BE ALREADY WORD-SEGMENTED!
line = "Tôi là <mask> trường đại_học Công_nghệ ."

input_ids = torch.tensor([tokenizer.encode(line)])



AttributeError: 'BaseModelOutputWithPoolingAndCrossAttentions' object has no attribute 'logits'

In [59]:
with torch.no_grad():
    features = phobert(input_ids)  # Models outputs are now tuples

# Extract the logits


Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Device set to use mps:0


[{'score': 0.51025390625,
  'token': 3716,
  'token_str': 'Hiệu_trưởng',
  'sequence': 'Tôi là Hiệu_trưởng trường đại_học Công_nghệ .'},
 {'score': 0.2998046875,
  'token': 4856,
  'token_str': 'hiệu_trưởng',
  'sequence': 'Tôi là hiệu_trưởng trường đại_học Công_nghệ .'},
 {'score': 0.039947509765625,
  'token': 3466,
  'token_str': 'giảng_viên',
  'sequence': 'Tôi là giảng_viên trường đại_học Công_nghệ .'},
 {'score': 0.03753662109375,
  'token': 250,
  'token_str': 'Chủ_tịch',
  'sequence': 'Tôi là Chủ_tịch trường đại_học Công_nghệ .'},
 {'score': 0.0199432373046875,
  'token': 649,
  'token_str': 'sinh_viên',
  'sequence': 'Tôi là sinh_viên trường đại_học Công_nghệ .'},
 {'score': 0.00891876220703125,
  'token': 1623,
  'token_str': 'chủ_tịch',
  'sequence': 'Tôi là chủ_tịch trường đại_học Công_nghệ .'},
 {'score': 0.0084381103515625,
  'token': 693,
  'token_str': 'Giám_đốc',
  'sequence': 'Tôi là Giám_đốc trường đại_học Công_nghệ .'},
 {'score': 0.007450103759765625,
  'token': 33

In [67]:
from pyvi import ViTokenizer, ViPosTagger

ViTokenizer.tokenize(u"Trường đại học bách khoa hà nội")

'Trường đại_học bách_khoa hà_nội'

In [96]:
# replace uỷ with ủy
dict_map = {
    "òa": "oà",
    "Òa": "Oà",
    "ÒA": "OÀ",
    "óa": "oá",
    "Óa": "Oá",
    "ÓA": "OÁ",
    "ỏa": "oả",
    "Ỏa": "Oả",
    "ỎA": "OẢ",
    "õa": "oã",
    "Õa": "Oã",
    "ÕA": "OÃ",
    "ọa": "oạ",
    "Ọa": "Oạ",
    "ỌA": "OẠ",
    "òe": "oè",
    "Òe": "Oè",
    "ÒE": "OÈ",
    "óe": "oé",
    "Óe": "Oé",
    "ÓE": "OÉ",
    "ỏe": "oẻ",
    "Ỏe": "Oẻ",
    "ỎE": "OẺ",
    "õe": "oẽ",
    "Õe": "Oẽ",
    "ÕE": "OẼ",
    "ọe": "oẹ",
    "Ọe": "Oẹ",
    "ỌE": "OẸ",
    "ùy": "uỳ",
    "Ùy": "Uỳ",
    "ÙY": "UỲ",
    "úy": "uý",
    "Úy": "Uý",
    "ÚY": "UÝ",
    "ủy": "uỷ",
    "Ủy": "Uỷ",
    "ỦY": "UỶ",
    "ũy": "uỹ",
    "Ũy": "Uỹ",
    "ŨY": "UỸ",
    "ụy": "uỵ",
    "Ụy": "Uỵ",
    "ỤY": "UỴ",
    }


In [ ]:
TXT = "Trung tâm Dự báo Khí tượng Thủy văn quốc gia cho biết lúc 7h hôm na, áp thấp nhiệt đới mạnh 61 km/h, cấp 6-7,"
# replace uỷ with ủy
dict_map = {
    "òa": "oà",
    "Òa": "Oà",
    "ÒA": "OÀ",
    "óa": "oá",
    "Óa": "Oá",
    "ÓA": "OÁ",
    "ỏa": "oả",
    "Ỏa": "Oả",
    "ỎA": "OẢ",
    "õa": "oã",
    "Õa": "Oã",
    "ÕA": "OÃ",
    "ọa": "oạ",
    "Ọa": "Oạ",
    "ỌA": "OẠ",
    "òe": "oè",
    "Òe": "Oè",
    "ÒE": "OÈ",
    "óe": "oé",
    "Óe": "Oé",
    "ÓE": "OÉ",
    "ỏe": "oẻ",
    "Ỏe": "Oẻ",
    "ỎE": "OẺ",
    "õe": "oẽ",
    "Õe": "Oẽ",
    "ÕE": "OẼ",
    "ọe": "oẹ",
    "Ọe": "Oẹ",
    "ỌE": "OẸ",
    "ùy": "uỳ",
    "Ùy": "Uỳ",
    "ÙY": "UỲ",
    "úy": "uý",
    "Úy": "Uý",
    "ÚY": "UÝ",
    "ủy": "uỷ",
    "Ủy": "Uỷ",
    "ỦY": "UỶ",
    "ũy": "uỹ",
    "Ũy": "Uỹ",
    "ŨY": "UỸ",
    "ụy": "uỵ",
    "Ụy": "Uỵ",
    "ỤY": "UỴ",
    }
from pyvi import ViTokenizer, ViPosTagger

ViTokenizer.tokenize(u"Trường đại học bách khoa hà nội")
import torch
from transformers import AutoModel, AutoTokenizer

phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
def process_text(text):
    for key, value in dict_map.items():
        text = text.replace(key, value)
    return text
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False
def sentence_prediction(text, top_k=30):
    text = ViTokenizer.tokenize(TXT)    
    
    error_indices = []
    for i in range(len(text.split())):
        text = process_text(text)
        new_list = text.split()
        original_word = new_list[i]
        if is_number(original_word):
            continue
        
        new_list[i] = tokenizer.mask_token
        masked_text = " ".join(new_list)
        start_time = time.time()
        predictions = pipeline(masked_text, top_k=100)
        topk = []
        for item in predictions:
            if 'token_str' in item:
                topk.append(item['token_str'])
        end_time = time.time()
        if original_word not in topk:
            print(f"Original Word: {original_word}")
            print(f"Masked Text: {masked_text}")
            print(f"->Top 10 Predictions: {topk}")
            error_indices.append((i, original_word, topk[0]))
        # print(f"Masked Text: {masked_text}")
    return error_indices

   

In [6]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)
example = """
Trước đy, giá đất ở khu vực này chỉ 2,7 triệu đồng mỗi m2.
"""
ner_results = nlp(example)
example_tok = ""
for e in ner_results:
    if "##" in e["word"]:
        example_tok = example_tok + e["word"].replace("##","")
    elif e["entity"] =="I":
        example_tok = example_tok + "_" + e["word"]
    else:
        example_tok = example_tok + " " + e["word"]
print(example_tok)


Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


 Trước đy , giá đất ở khu_vực này chỉ 2 , 7 triệu đồng mỗi m2 .


In [7]:
class VietnameseWordSegmenter:
    """
    A Vietnamese word segmentation class using NlpHUST/vi-word-segmentation model
    """
    
    def __init__(self, model_name="NlpHUST/vi-word-segmentation"):
        """
        Initialize the Vietnamese word segmenter
        
        Args:
            model_name (str): The pre-trained model name from HuggingFace
        """
        from transformers import AutoTokenizer, AutoModelForTokenClassification
        from transformers import pipeline
        
        self.model_name = model_name
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForTokenClassification.from_pretrained(model_name)
        self.nlp = pipeline("token-classification", model=self.model, tokenizer=self.tokenizer)
        
    def segment_text(self, text):
        """
        Segment Vietnamese text into words
        
        Args:
            text (str): Input Vietnamese text to be segmented
            
        Returns:
            str: Segmented text with underscores connecting compound words
        """
        # Clean the input text
        text = text.strip()
        if not text:
            return ""
            
        # Get NER results
        ner_results = self.nlp(text)
        
        # Process the results to create segmented text
        segmented_text = ""
        for entity in ner_results:
            word = entity["word"]
            entity_label = entity["entity"]
            
            if "##" in word:
                # Handle subword tokens (BERT tokenization artifacts)
                segmented_text += word.replace("##", "")
            elif entity_label == "I":
                # Inside entity - connect with underscore
                segmented_text += "_" + word
            else:
                # Beginning of entity or outside entity - add space
                segmented_text += " " + word
                
        return segmented_text.strip()
    
    def segment_sentences(self, sentences):
        """
        Segment multiple sentences
        
        Args:
            sentences (list): List of Vietnamese sentences
            
        Returns:
            list: List of segmented sentences
        """
        if isinstance(sentences, str):
            sentences = [sentences]
            
        segmented_sentences = []
        for sentence in sentences:
            segmented = self.segment_text(sentence)
            segmented_sentences.append(segmented)
            
        return segmented_sentences
    
    def __call__(self, text):
        """
        Allow the class to be called directly
        
        Args:
            text (str or list): Input text(s) to segment
            
        Returns:
            str or list: Segmented text(s)
        """
        if isinstance(text, list):
            return self.segment_sentences(text)
        else:
            return self.segment_text(text)

# Example usage:
if __name__ == "__main__":
    # Initialize the segmenter
    segmenter = VietnameseWordSegmenter()
    
    # Test with example text
    example_text = "Trước đy, giá đất ở khu vực này chỉ 2,7 triệu đồng mỗi m2."
    segmented_result = segmenter.segment_text(example_text)
    print(f"Original: {example_text}")
    print(f"Segmented: {segmented_result}")
    
    # Test with multiple sentences
    sentences = [
        "Trường đại học bách khoa hà nội",
        "Thủ tướng Chính phủ Phạm Minh Chính",
        "Ngân hàng nhà nước Việt Nam"
    ]
    
    segmented_sentences = segmenter.segment_sentences(sentences)
    for i, (original, segmented) in enumerate(zip(sentences, segmented_sentences)):
        print(f"\nSentence {i+1}:")
        print(f"Original: {original}")
        print(f"Segmented: {segmented}")

Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Original: Trước đy, giá đất ở khu vực này chỉ 2,7 triệu đồng mỗi m2.
Segmented: Trước đy , giá đất ở khu_vực này chỉ 2 , 7 triệu đồng mỗi m2 .

Sentence 1:
Original: Trường đại học bách khoa hà nội
Segmented: Trường đại_học bách_khoa hà_nội

Sentence 2:
Original: Thủ tướng Chính phủ Phạm Minh Chính
Segmented: Thủ_tướng Chính_phủ Phạm_Minh_Chính

Sentence 3:
Original: Ngân hàng nhà nước Việt Nam
Segmented: Ngân_hàng nhà_nước Việt_Nam


In [12]:
# Flask API Interface
from flask import Flask, request, jsonify, render_template_string
from flask_cors import CORS
import json

# Initialize Flask app
app = Flask(__name__)
CORS(app)

# Initialize the segmenter (global instance)
segmenter = None

def initialize_segmenter():
    """Initialize the segmenter if not already initialized"""
    global segmenter
    if segmenter is None:
        print("Initializing Vietnamese Word Segmenter...")
        segmenter = VietnameseWordSegmenter()
        print("Segmenter initialized successfully!")

@app.route('/api/segment', methods=['POST'])
def segment_text_api():
    """
    API endpoint to segment Vietnamese text
    
    Expected JSON payload:
    {
        "text": "Your Vietnamese text here"
    }
    
    Returns:
    {
        "original": "original text",
        "segmented": "segmented text",
        "status": "success"
    }
    """
    try:
        # Initialize segmenter if needed
        initialize_segmenter()
        
        # Get JSON data from request
        data = request.get_json()
        
        if not data or 'text' not in data:
            return jsonify({
                'error': 'No text provided. Please send JSON with "text" field.',
                'status': 'error'
            }), 400
        
        input_text = data['text'].strip()
        
        if not input_text:
            return jsonify({
                'error': 'Empty text provided.',
                'status': 'error'
            }), 400
        
        # Perform segmentation
        segmented_text = segmenter.segment_text(input_text)
        
        # Return results
        return jsonify({
            'original': input_text,
            'segmented': segmented_text,
            'status': 'success'
        })
    
    except Exception as e:
        return jsonify({
            'error': f'An error occurred: {str(e)}',
            'status': 'error'
        }), 500

@app.route('/api/segment/batch', methods=['POST'])
def segment_batch_api():
    """
    API endpoint to segment multiple Vietnamese sentences
    
    Expected JSON payload:
    {
        "texts": ["sentence 1", "sentence 2", ...]
    }
    """
    try:
        initialize_segmenter()
        
        data = request.get_json()
        
        if not data or 'texts' not in data:
            return jsonify({
                'error': 'No texts provided. Please send JSON with "texts" field.',
                'status': 'error'
            }), 400
        
        input_texts = data['texts']
        
        if not isinstance(input_texts, list):
            return jsonify({
                'error': 'texts field must be a list of strings.',
                'status': 'error'
            }), 400
        
        # Perform batch segmentation
        segmented_texts = segmenter.segment_sentences(input_texts)
        
        # Return results
        return jsonify({
            'original': input_texts,
            'segmented': segmented_texts,
            'status': 'success'
        })
    
    except Exception as e:
        return jsonify({
            'error': f'An error occurred: {str(e)}',
            'status': 'error'
        }), 500

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    return jsonify({
        'status': 'healthy',
        'service': 'Vietnamese Word Segmenter API',
        'segmenter_initialized': segmenter is not None
    })

@app.route('/')
def web_interface():
    """Web interface for the segmenter"""
    return render_template_string(WEB_TEMPLATE)

# Web interface HTML template
WEB_TEMPLATE = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Vietnamese Word Segmenter</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f5f5f5;
        }
        .container {
            background: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        h1 {
            color: #333;
            text-align: center;
            margin-bottom: 30px;
        }
        .input-group {
            margin-bottom: 20px;
        }
        label {
            display: block;
            margin-bottom: 8px;
            font-weight: bold;
            color: #555;
        }
        textarea {
            width: 100%;
            padding: 12px;
            border: 2px solid #ddd;
            border-radius: 5px;
            font-size: 14px;
            font-family: 'Courier New', monospace;
            resize: vertical;
            min-height: 150px;
            box-sizing: border-box;
        }
        textarea:focus {
            border-color: #4CAF50;
            outline: none;
        }
        .button-group {
            text-align: center;
            margin: 20px 0;
        }
        button {
            background-color: #4CAF50;
            color: white;
            padding: 12px 30px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
            margin: 0 10px;
        }
        button:hover {
            background-color: #45a049;
        }
        button:disabled {
            background-color: #ccc;
            cursor: not-allowed;
        }
        .clear-btn {
            background-color: #f44336;
        }
        .clear-btn:hover {
            background-color: #da190b;
        }
        .output-area {
            background-color: #f8f8f8;
            border: 2px solid #eee;
            border-radius: 5px;
        }
        .status {
            text-align: center;
            padding: 10px;
            margin: 10px 0;
            border-radius: 5px;
        }
        .status.success {
            background-color: #d4edda;
            color: #155724;
        }
        .status.error {
            background-color: #f8d7da;
            color: #721c24;
        }
        .status.loading {
            background-color: #cce7ff;
            color: #004085;
        }
        .example-texts {
            background-color: #e7f3ff;
            padding: 15px;
            border-radius: 5px;
            margin-bottom: 20px;
        }
        .example-text {
            cursor: pointer;
            color: #0066cc;
            text-decoration: underline;
            margin: 5px 0;
            display: block;
        }
        .example-text:hover {
            color: #004499;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🇻🇳 Vietnamese Word Segmenter</h1>
        
        <div class="example-texts">
            <h3>📝 Example texts (click to use):</h3>
            <span class="example-text" onclick="setInputText('Trường đại học bách khoa hà nội')">Trường đại học bách khoa hà nội</span>
            <span class="example-text" onclick="setInputText('Thủ tướng Chính phủ Phạm Minh Chính')">Thủ tướng Chính phủ Phạm Minh Chính</span>
            <span class="example-text" onclick="setInputText('Ngân hàng nhà nước Việt Nam')">Ngân hàng nhà nước Việt Nam</span>
            <span class="example-text" onclick="setInputText('Trước đây, giá đất ở khu vực này chỉ 2,7 triệu đồng mỗi m2.')">Trước đây, giá đất ở khu vực này chỉ 2,7 triệu đồng mỗi m2.</span>
        </div>
        
        <div class="input-group">
            <label for="inputText">Original Text:</label>
            <textarea id="inputText" placeholder="Enter Vietnamese text here..."></textarea>
        </div>
        
        <div class="button-group">
            <button onclick="segmentText()">🔍 Segment Text</button>
            <button class="clear-btn" onclick="clearText()">🗑️ Clear All</button>
        </div>
        
        <div id="status"></div>
        
        <div class="input-group">
            <label for="outputText">Segmented Text:</label>
            <textarea id="outputText" class="output-area" readonly placeholder="Segmented text will appear here..."></textarea>
        </div>
    </div>

    <script>
        function setInputText(text) {
            document.getElementById('inputText').value = text;
        }
        
        function clearText() {
            document.getElementById('inputText').value = '';
            document.getElementById('outputText').value = '';
            document.getElementById('status').innerHTML = '';
        }
        
        function showStatus(message, type) {
            const statusDiv = document.getElementById('status');
            statusDiv.innerHTML = message;
            statusDiv.className = `status ${type}`;
        }
        
        async function segmentText() {
            const inputText = document.getElementById('inputText').value.trim();
            const outputText = document.getElementById('outputText');
            
            if (!inputText) {
                showStatus('Please enter some text to segment.', 'error');
                return;
            }
            
            // Show loading status
            showStatus('Processing text...', 'loading');
            
            try {
                const response = await fetch('/api/segment', {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify({
                        text: inputText
                    })
                });
                
                const result = await response.json();
                
                if (result.status === 'success') {
                    outputText.value = result.segmented;
                    showStatus('✅ Text segmented successfully!', 'success');
                } else {
                    showStatus(`❌ Error: ${result.error}`, 'error');
                    outputText.value = '';
                }
                
            } catch (error) {
                showStatus(`❌ Error: ${error.message}`, 'error');
                outputText.value = '';
            }
        }
        
        // Allow Enter key to trigger segmentation (with Shift+Enter for new line)
        document.getElementById('inputText').addEventListener('keydown', function(event) {
            if (event.key === 'Enter' && !event.shiftKey) {
                event.preventDefault();
                segmentText();
            }
        });
    </script>
</body>
</html>
'''

# Function to run the server
def run_server(host='0.0.0.0', port=5000, debug=True):
    """
    Run the Flask server
    
    Args:
        host (str): Host address
        port (int): Port number
        debug (bool): Enable debug mode
    """
    print(f"Starting Vietnamese Word Segmenter API server...")
    print(f"Web interface will be available at: http://localhost:{port}")
    print(f"API endpoint: http://localhost:{port}/api/segment")
    
    app.run(host=host, port=port, debug=debug)

# Example of how to run the server
print("Vietnamese Word Segmenter API is ready!")
print("To start the server, run: run_server()")
print("Or run: app.run(host='0.0.0.0', port=5000, debug=True)")

Vietnamese Word Segmenter API is ready!
To start the server, run: run_server()
Or run: app.run(host='0.0.0.0', port=5000, debug=True)


In [ ]:
# Start the server
# Note: This will run the server and block the cell execution
# The web interface will be available at http://localhost:5000
# API endpoint will be available at http://localhost:5000/api/segment

if __name__ == "__main__":
    # Run the server
    run_server(host='0.0.0.0', port=5000, debug=False)

In [9]:
class VietnameseWordSegmenter:
    """
    A class for Vietnamese word segmentation using transformers.
    
    This class provides methods to segment Vietnamese text into words,
    handling subword tokens and word boundaries properly.
    """
    
    def __init__(self, model_name="NlpHUST/vi-word-segmentation"):
        """
        Initialize the Vietnamese Word Segmenter.
        
        Args:
            model_name (str): The name of the pre-trained model to use.
                            Default is "NlpHUST/vi-word-segmentation"
        """
        print(f"Loading Vietnamese Word Segmentation model: {model_name}")
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.model = AutoModelForTokenClassification.from_pretrained(model_name)
        self.pipeline = pipeline("token-classification", 
                                model=self.model, 
                                tokenizer=self.tokenizer)
        print("Model loaded successfully!")
    
    def segment(self, text):
        """
        Segment Vietnamese text into words.
        
        Args:
            text (str): The Vietnamese text to segment
            
        Returns:
            str: The segmented text with words separated by spaces and 
                 compound words connected by underscores
        """
        if not text or not text.strip():
            return ""
        
        # Get token classification results
        ner_results = self.pipeline(text.strip())
        
        # Process the results to create segmented text
        segmented_text = self._process_tokens(ner_results)
        
        return segmented_text.strip()
    
    def _process_tokens(self, ner_results):
        """
        Process the token classification results to create segmented text.
        
        Args:
            ner_results (list): List of token classification results
            
        Returns:
            str: Processed segmented text
        """
        segmented_text = ""
        
        for token_info in ner_results:
            word = token_info["word"]
            entity = token_info["entity"]
            
            if "##" in word:
                # Handle subword tokens (remove ## prefix)
                segmented_text += word.replace("##", "")
            elif entity == "I":
                # Inside entity - connect with underscore
                segmented_text += "_" + word
            else:
                # Beginning of entity or outside entity - add space
                segmented_text += " " + word
        
        return segmented_text
    
    def segment_batch(self, texts):
        """
        Segment multiple texts at once.
        
        Args:
            texts (list): List of Vietnamese texts to segment
            
        Returns:
            list: List of segmented texts
        """
        return [self.segment(text) for text in texts]
    
    def get_word_boundaries(self, text):
        """
        Get detailed word boundary information.
        
        Args:
            text (str): The Vietnamese text to analyze
            
        Returns:
            list: List of dictionaries containing word information
        """
        if not text or not text.strip():
            return []
        
        ner_results = self.pipeline(text.strip())
        word_info = []
        current_word = ""
        current_start = 0
        
        for i, token_info in enumerate(ner_results):
            word = token_info["word"]
            entity = token_info["entity"]
            start = token_info.get("start", 0)
            end = token_info.get("end", 0)
            
            if "##" in word:
                current_word += word.replace("##", "")
            elif entity == "I":
                current_word += "_" + word
            else:
                # Save previous word if exists
                if current_word:
                    word_info.append({
                        "word": current_word.strip(),
                        "start": current_start,
                        "end": end
                    })
                
                # Start new word
                current_word = word
                current_start = start
        
        # Add the last word
        if current_word:
            word_info.append({
                "word": current_word.strip(),
                "start": current_start,
                "end": ner_results[-1].get("end", len(text)) if ner_results else len(text)
            })
        
        return word_info
    
    def __call__(self, text):
        """
        Make the class callable - same as segment method.
        
        Args:
            text (str): The Vietnamese text to segment
            
        Returns:
            str: The segmented text
        """
        return self.segment(text)

In [20]:
# Example usage of the VietnameseWordSegmenter class

# Initialize the segmenter
segmenter = VietnameseWordSegmenter()

# Example texts to segment
examples = [
    "Theo bảng giá đất hiện hành, đất ở hẻm sâu ở đường Nguyễn Xiển có giá 28 triệu đồng mỗi m2, trong khi đất nông nghiệp chỉ khoảng 600.000 đồng, chênh hơn 27 triệu đồng mỗi m2, khiến chi phí chuyển đổi tăng trên 10 lần. Trước đây, giá đất ở khu vực này chỉ 2,7 triệu đồng mỗi m2.",
    "Không riêng chị Hạnh, nhiều người dân phải rao bán nhà đất giá rẻ do không kham nổi nghĩa vụ tài chính khi chuyển mục đích. Ông Lê Văn Quyết (phường Long Bình) có thửa đất 210 m2 đủ điều kiện chuyển thổ cư tại phường Long Bình, TP HCM nhưng vướng quy hoạch. Khi được gỡ vướng, ông Quyết tìm hiểu thủ tục tách thửa và chuyển mục đích thì rất sốc vì phải đóng hơn 5,7 tỷ đồng tiền sử dụng đất, vượt khả năng chi trả.",
    "Trường đại học bách khoa hà nội là một trong những trường hàng đầu"
]

print("=== Vietnamese Word Segmentation Demo ===\n")

for i, text in enumerate(examples, 1):
    print(f"Example {i}:")
    print(f"Original: {text}")
    
    # Basic segmentation
    segmented = segmenter.segment(text)
    print(f"Segmented: {segmented}")
    
    # Get word boundaries (optional)
    boundaries = segmenter.get_word_boundaries(text)
    print(f"Word boundaries: {len(boundaries)} words found")
    
    print("-" * 50)

# Batch processing example
print("\n=== Batch Processing ===")
batch_results = segmenter.segment_batch(examples)
for i, result in enumerate(batch_results, 1):
    print(f"Batch {i}: {result}")

# Using the class as a callable
print("\n=== Using as Callable ===")
callable_result = segmenter("Tôi yêu Việt Nam")
print(f"Callable result: {callable_result}")

Loading Vietnamese Word Segmentation model: NlpHUST/vi-word-segmentation


Device set to use mps:0
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Model loaded successfully!
=== Vietnamese Word Segmentation Demo ===

Example 1:
Original: Theo bảng giá đất hiện hành, đất ở hẻm sâu ở đường Nguyễn Xiển có giá 28 triệu đồng mỗi m2, trong khi đất nông nghiệp chỉ khoảng 600.000 đồng, chênh hơn 27 triệu đồng mỗi m2, khiến chi phí chuyển đổi tăng trên 10 lần. Trước đây, giá đất ở khu vực này chỉ 2,7 triệu đồng mỗi m2.
Segmented: Theo bảng giá đất hiện_hành , đất ở hẻm sâu ở đường Nguyễn_Xiển có_giá 28 triệu đồng mỗi m2 , trong khi đất nông_nghiệp chỉ khoảng 600 . 000 đồng , chênh hơn 27 triệu đồng mỗi m2 , khiến chi_phí chuyển_đổi tăng trên 10 lần . Trước_đây , giá đất ở khu_vực này chỉ 2 , 7 triệu đồng mỗi m2 .
Word boundaries: 63 words found
--------------------------------------------------
Example 2:
Original: Không riêng chị Hạnh, nhiều người dân phải rao bán nhà đất giá rẻ do không kham nổi nghĩa vụ tài chính khi chuyển mục đích. Ông Lê Văn Quyết (phường Long Bình) có thửa đất 210 m2 đủ điều kiện chuyển thổ cư tại phường Long Bình,